In [1]:
from GeneralTools.misc_fun import FLAGS

FLAGS.TENSORFLOW_VERSION


'1.13.0'

In [3]:
import tensorflow as tf
import numpy as np

# load the data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
print(x_train.shape)
print(y_train.shape)
print('x_train range: {} - {}'.format(np.amin(x_train), np.amax(x_train)))
print('y_train range: {} - {}'.format(np.amin(y_train), np.amax(y_train)))


(60000, 28, 28)
(60000,)
x_train range: 0 - 255
y_train range: 0 - 9


In [5]:
import tensorflow as tf
import numpy as np
import os.path
from GeneralTools.misc_fun import FLAGS
"""
Here we show how to save data to tfrecords
"""


# define macro
# FloatList, Int64List and BytesList are three base feature types
def _float_feature(value):
    # Returns a float_list from a float / double.
    return tf.train.Feature(
        float_list=tf.train.FloatList(value=[value])
        if isinstance(value, float) else tf.train.FloatList(value=value))


def _int64_feature(value):
    # Returns an int64_list from a bool / enum / int / uint
    # numpy int is not int!
    return tf.train.Feature(
        int64_list=tf.train.Int64List(value=[value])
        if isinstance(value, int) else tf.train.Int64List(value=value))


def _bytes_feature(value):
    # Returns a bytes_list from a string / byte.
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[value])
        if isinstance(value, (str, bytes)) else tf.train.BytesList(value=value))


file_path = os.path.join(FLAGS.DEFAULT_IN, 'fashionmnist', 'fashionmnist_train.tfrecords')
if not os.path.exists(file_path):
    writer = tf.python_io.TFRecordWriter(file_path)
    x_train = np.reshape(x_train, (x_train.shape[0], -1))  # x_train shape [6000, 784]
    """
    In case the data is too large, we need to read them one-by-one and 
    save them to multiple tfrecord files. Optimally, each tfrecord file 
    may be around 100 MB (no larger than 1 GB for faster parallel reading)
    """
    for image_flat, label in zip(x_train, y_train):
        instance = tf.train.Example(features=tf.train.Features(feature={
            'x': _bytes_feature(image_flat.tobytes()),
            'y': _int64_feature(int(label))  # numpy int is not int!
        }))
        writer.write(instance.SerializeToString())
    
    writer.close()  # or put everything under: with ... as writer:
    print('Writing to tfrecords file finished.')
else:
    print('File {} already exists.'.format(file_path))


File C:/Users/richa/PycharmProjects/ai_mechatronics/Datasets/fashionmnist\fashionmnist_train.tfrecords already exists.


In [13]:
import tensorflow as tf
from GeneralTools.misc_fun import FLAGS
"""
Here we show how to read data from tfrecords
"""
# possible dtypes are tf.float32, tf.int64 and tf.string, 
# depending on how we save the data
x_dtype = tf.string
y_dtype = tf.int64
num_samples = 60000
num_features = 784  # 28x28
num_labels = 1


def my_parser(example_proto):
    """ This function parses a single datum
    
    :param example_proto: 
    :return: 
    """
    # parse a single example
    # if tf.string, the length of features is [] (that is, 1)
    x_config = tf.FixedLenFeature([], tf.string)\
        if x_dtype == tf.string else tf.FixedLenFeature([num_features], x_dtype)
    y_config = tf.FixedLenFeature([], tf.string)\
        if y_dtype == tf.string else tf.FixedLenFeature([num_labels], y_dtype)
    proto_config = {'x': x_config, 'y': y_config}
    datum = tf.parse_single_example(example_proto, features=proto_config)
    
    # cast datum to data types suitable for downstream processing
    if x_dtype == tf.string:
        # first decode data to uint8, as data is stored in this way
        datum['x'] = tf.decode_raw(datum['x'], tf.uint8)
        # then cast data to tf.float32
        datum['x'] = tf.cast(datum['x'], tf.float32)
    if y_dtype == tf.string:
        # avoid using string labels like 'cat', 'dog', use integers instead
        datum['y'] = tf.decode_raw(datum['y'], tf.uint8)
        datum['y'] = tf.cast(datum['y'], tf.int32)
    if y_dtype == tf.int64:
        datum['y'] = tf.cast(datum['y'], tf.int32)
        
    # do the following pre-processing
    # since we know x is image, we could reshape it here
    # otherwise, we could always add dataset.map somewhere later
    datum['x'] = tf.divide(datum['x'], 255.0)
    if FLAGS.IMAGE_FORMAT is 'channels_first':
        datum['x'] = tf.reshape(datum['x'], (1, 28, 28))
    else:
        datum['x'] = tf.reshape(datum['x'], (28, 28, 1))
        
    return datum['x'], datum['y']
    

file_path = os.path.join(FLAGS.DEFAULT_IN, 'fashionmnist', 'fashionmnist_train.tfrecords')
dataset = tf.data.TFRecordDataset(file_path)
"""
My laptop have 4 threads so I set num_parallel_calls to 4. 
Yours may have 8 or more, depending on the CPU.
"""
dataset = dataset.map(my_parser, num_parallel_calls=4)

"""
The rest is similar to the previous example of loading mnist in
TensorFlow_basics_02_session_and_input.ipynb
"""
batch_size = 64
num_epoch = None
dataset = dataset.skip(num_samples // batch_size)
dataset = dataset.shuffle(5000)
dataset = dataset.batch(batch_size)
dataset = dataset.repeat(num_epoch)

iterator = dataset.make_one_shot_iterator()
x_batch, y_batch = iterator.get_next()
print(x_batch)
print(y_batch)

"""
If the downstream requires the first dimension to be known,
we can manually set a shape here. But use it carefully, as:
1. this will not change the actual shape of x_batch, y_batch
2. if there is inconsistency in shape during runtime, it may
cause error. 
"""
x_batch.set_shape((batch_size, 1, 28, 28))
y_batch.set_shape((batch_size, 1))
print(x_batch)
print(y_batch)

with tf.Session() as sess:
    xv, yv = sess.run([x_batch, y_batch])
    print(xv.shape)
    print(yv.shape)


Tensor("IteratorGetNext_6:0", shape=(?, 1, 28, 28), dtype=float32)
Tensor("IteratorGetNext_6:1", shape=(?, 1), dtype=int32)
Tensor("IteratorGetNext_6:0", shape=(64, 1, 28, 28), dtype=float32)
Tensor("IteratorGetNext_6:1", shape=(64, 1), dtype=int32)


(64, 1, 28, 28)
(64, 1)
